In [31]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, \
f1_score, log_loss

import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification,  \
Trainer, TrainingArguments, TextClassificationPipeline

import evaluate
from scipy.special import softmax
import warnings
from warnings import simplefilter

import json
import yaml

import tqdm
from tqdm import tqdm_notebook
from tqdm.auto import tqdm
import shap
from typing import Dict

warnings.filterwarnings("ignore")
simplefilter("ignore", category=RuntimeWarning)

In [2]:
config_path = "../config/config.yaml"
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

In [3]:
preproc = config['preprocessing']
train_params = config['train']
test_params = config['test']

In [4]:
def get_dataset(path: str) -> pd.DataFrame:
    """
    Загрузка датасета
    :param: path - путь к файлу
    :return: data: датафрейм
    """

    data = pd.read_csv(path)
     
    return data


def plotting_trainer_loss(trainer):
    auc = []
    eval_loss = []
    
    
    for step in trainer.state.log_history:
        try:
            auc.append(step["eval_roc_auc"])
            eval_loss.append(step['eval_loss'])
            
        except KeyError:
            continue
    
    fig, axes = plt.subplots(ncols=2, figsize=(15, 5))
    sns.lineplot(eval_loss, ax=axes[0], color='orange')
    sns.lineplot(auc, ax=axes[1])
    
    axes[0].set_title('Validation Loss')
    axes[1].set_title('ROC-AUC SCORE')
    axes[0].set(xlabel='Epochs')
    axes[1].set(xlabel='Epochs')
    
    
def compute_metrics(eval_preds):
    """Расчет метрики roc-auc"""

    metric = evaluate.load("roc_auc")
    logits, labels = eval_preds
    predictions = softmax(logits)[:, 1]
    res = metric.compute(prediction_scores=predictions, 
                         references=labels)

    return {'roc_auc': res['roc_auc']}


def get_metrics(y_test: np.array, y_pred: np.array, y_proba: np.array) -> Dict:
    dict_metrics = {
        'roc_auc': round(roc_auc_score(y_test, y_proba[:, 1]), 3),
        'precision': round(precision_score(y_test, y_pred), 3),
        'recall': round(recall_score(y_test, y_pred), 3),
        'f1': round(f1_score(y_test, y_pred), 3),
        'logloss': round(log_loss(y_test, y_proba), 3)
    }

    return dict_metrics

In [5]:
class PrepareData:
    """
    Представление текстов с помощью bert-эмбеддингов
    """
    def __init__(self, texts, tokenizer, batch_size_split=train_params['batch_size_split'], 
                 max_length=train_params['max_length']):
        
        self.texts = texts
        self.tokenizer = tokenizer
        self.batch_size_split = batch_size_split
        self.max_length = max_length
        
    def pre_tokenizer(self, text):
        return self.tokenizer(text,
                              add_special_tokens=True, 
                              max_length=self.max_length,
                              pad_to_max_length=True,
                              truncation=True,
                              return_attention_mask=True,
                              return_tensors='pt')
    
        
    def transform(self):
        
        N = len(self.texts)
        size_split = N // self.batch_size_split

        train_encodings = self.pre_tokenizer(self.texts[:size_split])
        input_ids = train_encodings['input_ids']
        attention_mask = train_encodings['attention_mask']
        token_type_ids = train_encodings['token_type_ids']

        for pos in tqdm(range(size_split, N, size_split)):
            train_encodings_2 = self.pre_tokenizer(self.texts[pos:pos +
                                                              size_split])
            input_ids = torch.cat((input_ids, train_encodings_2['input_ids']))
            attention_mask = torch.cat(
                (attention_mask, train_encodings_2['attention_mask']))
            token_type_ids = torch.cat(
                (token_type_ids, train_encodings_2['token_type_ids']))
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids
        } 

In [6]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Класс представления датасета, содержащего объекты-эмбеддинги, поддерживающий
    итерацию для использования с объектом Trainer
    """

    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Если датасет содержит метки класса
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [14]:
config

{'preprocessing': {'random_state': 10,
  'test_size': 0.2,
  'raw_path': '../data/raw/data_reviews.csv',
  'train_path': '../data/processed/train_data.csv',
  'test_path': '../data/processed/test_data.csv',
  'scrape_path': '../data/raw/scrape_data.csv',
  'connection_path': '../config/connection.json'},
 'train': {'max_length': 512,
  'random_state': 10,
  'learning_rate': 2e-05,
  'tokenizer_path': 'cointegrated/rubert-tiny2',
  'model_path': 'cointegrated/rubert-tiny2',
  'trainer_path': '../models/history',
  'trainer_log_path': '../models/history/history.json',
  'epochs': 5,
  'weight_decay': 0.01,
  'per_device_batch_size': 64,
  'batch_size_split': 10,
  'metrics_path': '../report/metrics/metrics.json'},
 'test': {'model_path': '../models/bert-tiny2',
  'evaluate_path': '../data/check/test_data.csv'},
 'endpoints': {'train': 'http://fastapi:8000/train',
  'prediction_input': 'http://fastapi:8000/predict_from_input',
  'prediction_from_file': 'http://fastapi:8000/predict',
  'sc

In [16]:
test_df = get_dataset(test_params['evaluate_path'])

In [8]:
tokenizer = BertTokenizer.from_pretrained(train_params['tokenizer_path'])

In [9]:
clf = PrepareData(test_df.reviewText.tolist(), tokenizer)
test_encodings = clf.transform()

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
test_dataset = CustomDataset(test_encodings, test_df.target.tolist())

In [12]:
model = BertForSequenceClassification.from_pretrained(test_params['model_path'])

In [13]:
trainer = Trainer(
    model=model
)

# Предсказание модели классификации Bert
y_pred = trainer.predict(test_dataset)

# Получение вероятностей
pred_proba = y_pred[0]

# Получение предсказанных меток класса
pred = pred_proba.argmax(axis=1)

In [29]:
test_df['prediction'] = pred
test_df.head()

,reviewText,prediction
0,"с озоном познакомился давно , но поначалу как ...",1
1,знакома и пользуюсь услугами уже не первый год...,1
2,заказываю уже третий ноутбук в течении х месяц...,0
3,"здравствуйте , я давно уже на озоне приобретаю...",0
4,"мне очень не нравится , что озон требует паспо...",0


In [32]:
pipe = TextClassificationPipeline(model=model, 
                                  tokenizer=tokenizer, 
                                  return_all_scores=True)


def score_and_visualize(text: str):
    """
    Оценивает и визуализирует предсказания модели для входного текста с помощью
    библиотеки shap.

    Параметры:
    text: Входной текст для оценки и визуализации.
    """
    prediction = pipe([text])
    print(prediction[0])

    explainer = shap.Explainer(pipe)
    shap_values = explainer([text])

    shap.plots.text(shap_values)

In [35]:
score_and_visualize(test_df.reviewText[np.random.choice(len(test_df))])

[{'label': 'Negative', 'score': 0.1920837014913559}, {'label': 'Positive', 'score': 0.8079162836074829}]



На примере случайного отзывас помощью библиотеки shap можно оценить вклад словосочетайний в тональность: "цены адекватные на все самое необходимое" - вносит большой вклад в позитивную тональность отзыва.